In [1]:
# Importing modules

import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn
from PIL import Image

In [2]:
# creating NN and model

class TinyVGG(nn.Module):
    """
    Model architecture copying TinyVGG from: 
    https://poloclub.github.io/cnn-explainer/
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3, # how big is the square that's going over the image?
                      stride=1, # default
                      padding=1), # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number 
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from? 
            # It's because each layer of our network compresses and changes the shape of our inputs data.
            nn.Linear(in_features=hidden_units*16*16,
                      out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        # print(x.shape)
        x = self.conv_block_2(x)
        # print(x.shape)
        x = self.classifier(x)
        # print(x.shape)
        return x
        # return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion

model_0 = TinyVGG(input_shape=3,hidden_units=10,output_shape=2)
model_0.load_state_dict(torch.load("C:/Users/latik/OneDrive/Desktop/PROJECT/model_0.pth"))
model_0.eval()
labels = ["Plastic","No-plastic"]



In [3]:
# Image Prediction......

# path = "C:/Users/latik/OneDrive/Desktop/PROJECT/no-plastics.jpeg"

def predict(testing_img: Image):
    # print(img)
    # testing_img = Image.open(img)

    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(64),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(testing_img)
    input_batch = input_tensor.unsqueeze(0)

    with torch.inference_mode():
        
        pred=model_0(input_batch)

    pred= torch.softmax(pred, dim=1)
    pred_lbl= torch.argmax(pred)

    # image =  Image.open(path)
    return labels[pred_lbl]

In [4]:
import gradio as gr

interface = gr.Interface(
    fn=predict,
    inputs=[gr.Image(type="pil")],
    outputs=["text"],
)

interface.launch()

c:\Users\latik\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
